<a href="https://colab.research.google.com/github/ManuelRomero2025/CienciasdeDatos1/blob/main/datos_simulada_con_errores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Generar base de datos simulada con errores**



In [9]:
import pandas as pd
import numpy as np

np.random.seed(42)  # Para reproducibilidad

# Simular datos correctos
n = 100
data = {
    'id': range(1, n+1),
    'nombre': np.random.choice(['Ana', 'Luis', 'Carlos', 'María', 'Juan'], size=n),
    'edad': np.random.randint(18, 65, size=n),
    'ingresos_mensuales': np.random.normal(2000, 500, size=n).round(2),
    'fecha_ingreso': pd.date_range(start='2022-01-01', periods=n, freq='7D')
}

df = pd.DataFrame(data)

# 🔸 Introducir datos inconsistentes
df.loc[5, 'edad'] = -3                             # Edad negativa (inválida)
df.loc[10, 'nombre'] = '  ANA '                    # Nombre con espacios
df.loc[15, 'nombre'] = 'juan'                      # Nombre en minúscula
df.loc[20, 'nombre'] = 'CARLOS'                    # Nombre en mayúsculas
df.loc[25, 'ingresos_mensuales'] = '2.300,50'      # Separador decimal europeo
df.loc[30, 'ingresos_mensuales'] = '1900.80'       # Número como string
df.loc[35, 'fecha_ingreso'] = '15 enero 2022'      # Fecha en formato alternativo
df.loc[40, 'fecha_ingreso'] = '2022/31/12'         # Fecha mal escrita
df.loc[45, 'edad'] = 130                           # Edad muy alta (fuera de rango)
df.loc[50, 'edad'] = 'cuarenta'                    # Edad como texto
df.loc[55, 'ingresos_mensuales'] = 'Dos mil'       # Ingreso como texto
df.loc[60, 'ingresos_mensuales'] = np.nan          # Valor faltante

# Vista rápida
print(df.head(100))

print(df.head())
print(df.info())


   id  nombre edad ingresos_mensuales        fecha_ingreso
0   1   María   45            2716.76  2022-01-01 00:00:00
1   2    Juan   24             2532.9  2022-01-08 00:00:00
2   3  Carlos   26             750.62  2022-01-15 00:00:00
3   4    Juan   25            2740.16  2022-01-22 00:00:00
4   5    Juan   29            3079.36  2022-01-29 00:00:00
5   6    Luis   -3            2615.89  2022-02-05 00:00:00
6   7  Carlos   50            1892.84  2022-02-12 00:00:00
7   8  Carlos   40             1655.4  2022-02-19 00:00:00
8   9  Carlos   41            2363.92  2022-02-26 00:00:00
9  10    Juan   54            1690.47  2022-03-05 00:00:00
   id  nombre edad ingresos_mensuales        fecha_ingreso
0   1   María   45            2716.76  2022-01-01 00:00:00
1   2    Juan   24             2532.9  2022-01-08 00:00:00
2   3  Carlos   26             750.62  2022-01-15 00:00:00
3   4    Juan   25            2740.16  2022-01-22 00:00:00
4   5    Juan   29            3079.36  2022-01-29 00:00:

<ipython-input-9-6d49c167cf0c>:23: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2.300,50' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[25, 'ingresos_mensuales'] = '2.300,50'      # Separador decimal europeo
<ipython-input-9-6d49c167cf0c>:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '15 enero 2022' has dtype incompatible with datetime64[ns], please explicitly cast to a compatible dtype first.
  df.loc[35, 'fecha_ingreso'] = '15 enero 2022'      # Fecha en formato alternativo
<ipython-input-9-6d49c167cf0c>:28: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'cuarenta' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df.loc[50, 'edad'] = 'cuarenta'   

# Limpieza de la columna nombre

In [5]:
# Eliminar espacios y estandarizar en mayúscula inicial (title case)
df['nombre'] = df['nombre'].str.strip().str.lower().str.title()


# Limpieza de la columna edad

In [11]:
# Convertir a numérico (coerce convierte errores a NaN)
df['edad'] = pd.to_numeric(df['edad'], errors='coerce')

# Sustituir edades fuera del rango [18, 100] por NaN
df.loc[(df['edad'] < 18) | (df['edad'] > 100), 'edad'] = np.nan

# Imputar con la mediana de las edades válidas
mediana_edad = df['edad'].median()
df['edad'] = df['edad'].fillna(mediana_edad)



# Limpieza de la columna ingresos_mensuales

In [12]:
# Convertir todo a string, reemplazar coma decimal y convertir a float
df['ingresos_mensuales'] = df['ingresos_mensuales'].astype(str).str.replace(',', '.', regex=False)

# Convertir a numérico (errores serán NaN)
df['ingresos_mensuales'] = pd.to_numeric(df['ingresos_mensuales'], errors='coerce')

# Eliminar valores negativos o extremadamente bajos (si los hay)
df.loc[df['ingresos_mensuales'] < 0, 'ingresos_mensuales'] = np.nan

# Imputar ingresos faltantes con la media
media_ingresos = df['ingresos_mensuales'].mean()
df['ingresos_mensuales'] = df['ingresos_mensuales'].fillna(media_ingresos)



In [13]:
# Convertir fechas con errores a NaT (fecha inválida)
df['fecha_ingreso'] = pd.to_datetime(df['fecha_ingreso'], errors='coerce')

# Reemplazar fechas inválidas con la mediana de las fechas válidas
mediana_fecha = df['fecha_ingreso'].dropna().median()
df['fecha_ingreso'] = df['fecha_ingreso'].fillna(mediana_fecha)


In [19]:

print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id                  100 non-null    int64         
 1   nombre              100 non-null    object        
 2   edad                100 non-null    float64       
 3   ingresos_mensuales  100 non-null    float64       
 4   fecha_ingreso       100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(1)
memory usage: 4.0+ KB
None
   id  nombre  edad  ingresos_mensuales fecha_ingreso
0   1   María  45.0             2716.76    2022-01-01
1   2    Juan  24.0             2532.90    2022-01-08
2   3  Carlos  26.0              750.62    2022-01-15
3   4    Juan  25.0             2740.16    2022-01-22
4   5    Juan  29.0             3079.36    2022-01-29


In [18]:
print(df.head(100))

     id  nombre  edad  ingresos_mensuales fecha_ingreso
0     1   María  45.0             2716.76    2022-01-01
1     2    Juan  24.0             2532.90    2022-01-08
2     3  Carlos  26.0              750.62    2022-01-15
3     4    Juan  25.0             2740.16    2022-01-22
4     5    Juan  29.0             3079.36    2022-01-29
..  ...     ...   ...                 ...           ...
95   96  Carlos  45.0             1486.47    2023-10-28
96   97    Juan  40.0             2441.42    2023-11-04
97   98    Luis  54.0             1775.35    2023-11-11
98   99    Luis  49.0             2209.32    2023-11-18
99  100     Ana  50.0             1635.91    2023-11-25

[100 rows x 5 columns]


# **Ejemplo 2**

In [21]:
import pandas as pd
import numpy as np

np.random.seed(123)

n = 100
df = pd.DataFrame({
    'ID': range(1, n + 1),
    'Nombre': np.random.choice(['ana', ' Ana ', 'Luis', 'LUIS', 'María', 'maria'], n),
    'Edad': np.random.choice([25, 35, 60, 150, -5, 'treinta', np.nan], n),
    'Ingreso_Mensual': np.random.choice(['2000.50', '3.000,75', '-1500', 'Dos mil', np.nan, '2200.00'], n),
    'Genero': np.random.choice(['M', 'F', 'Masculino', 'Femenino', 'mujer', 'hombre'], n),
    'Fecha_Registro': np.random.choice(['2021-05-01', '01/06/2021', 'junio 2021', '2021/30/12', '15-07-2021', np.nan], n)
})

# Incluir duplicados
df = pd.concat([df, df.iloc[[5, 10]]], ignore_index=True)

print(df.head(100))

     ID Nombre Edad Ingreso_Mensual     Genero Fecha_Registro
0     1  maria   35        3.000,75     hombre            nan
1     2   Luis  nan        3.000,75          M     2021-05-01
2     3  María   -5         Dos mil          F     2021-05-01
3     4   Luis  nan         Dos mil  Masculino     2021/30/12
4     5   Ana   150           -1500  Masculino     01/06/2021
..  ...    ...  ...             ...        ...            ...
95   96   LUIS   35             nan     hombre     2021/30/12
96   97   LUIS  150         Dos mil   Femenino     junio 2021
97   98    ana   60           -1500      mujer     15-07-2021
98   99   Ana   150         2200.00          M     2021/30/12
99  100   LUIS  nan             nan     hombre     2021/30/12

[100 rows x 6 columns]


In [22]:
# =========================
# 1. Limpieza de columnas tipo texto
# =========================
df['Nombre'] = df['Nombre'].astype(str).str.strip().str.lower().str.title()

# =========================
# 2. Normalizar columna 'Genero'
# =========================
df['Genero'] = df['Genero'].replace({
    'M': 'Masculino',
    'F': 'Femenino',
    'hombre': 'Masculino',
    'mujer': 'Femenino'
}).str.title()

# =========================
# 3. Limpieza de la columna Edad
# =========================
df['Edad'] = pd.to_numeric(df['Edad'], errors='coerce')
df.loc[(df['Edad'] < 18) | (df['Edad'] > 100), 'Edad'] = np.nan
df['Edad'] = df['Edad'].fillna(df['Edad'].median())

# =========================
# 4. Limpieza de Ingreso_Mensual
# =========================
df['Ingreso_Mensual'] = df['Ingreso_Mensual'].astype(str).str.replace(',', '.')
df['Ingreso_Mensual'] = pd.to_numeric(df['Ingreso_Mensual'], errors='coerce')
df.loc[df['Ingreso_Mensual'] < 0, 'Ingreso_Mensual'] = np.nan
df['Ingreso_Mensual'] = df['Ingreso_Mensual'].fillna(df['Ingreso_Mensual'].mean())

# =========================
# 5. Conversión y corrección de fechas
# =========================
df['Fecha_Registro'] = pd.to_datetime(df['Fecha_Registro'], errors='coerce')
fecha_mediana = df['Fecha_Registro'].dropna().median()
df['Fecha_Registro'] = df['Fecha_Registro'].fillna(fecha_mediana)

# =========================
# 6. Eliminación de duplicados
# =========================
df = df.drop_duplicates()

# =========================
# 7. Validación y resumen
# =========================
print("\n✅ Información final del DataFrame limpio:")
print(df.info())
print(df.describe(include='all'))



✅ Información final del DataFrame limpio:
<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   ID               100 non-null    int64         
 1   Nombre           100 non-null    object        
 2   Edad             100 non-null    float64       
 3   Ingreso_Mensual  100 non-null    float64       
 4   Genero           100 non-null    object        
 5   Fecha_Registro   100 non-null    datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(1), object(2)
memory usage: 5.5+ KB
None
                ID Nombre        Edad  Ingreso_Mensual     Genero  \
count   100.000000    100  100.000000       100.000000        100   
unique         NaN      4         NaN              NaN          2   
top            NaN   Luis         NaN              NaN  Masculino   
freq           NaN     39         NaN              NaN         58   
mean

In [23]:
print(df.head(100))

     ID Nombre  Edad  Ingreso_Mensual     Genero Fecha_Registro
0     1  Maria  35.0      2103.467742  Masculino     2021-05-01
1     2   Luis  35.0      2103.467742  Masculino     2021-05-01
2     3  María  35.0      2103.467742   Femenino     2021-05-01
3     4   Luis  35.0      2103.467742  Masculino     2021-05-01
4     5    Ana  35.0      2103.467742  Masculino     2021-05-01
..  ...    ...   ...              ...        ...            ...
95   96   Luis  35.0      2103.467742  Masculino     2021-05-01
96   97   Luis  35.0      2103.467742   Femenino     2021-05-01
97   98    Ana  60.0      2103.467742   Femenino     2021-05-01
98   99    Ana  35.0      2200.000000  Masculino     2021-05-01
99  100   Luis  35.0      2103.467742  Masculino     2021-05-01

[100 rows x 6 columns]
